In [4]:
import os
import open3d as o3d
import numpy as np
import cv2
from PIL import Image
from sklearn.cluster import DBSCAN

In [20]:
def convert_v2f(file_path, fps=10):
    os.makedirs("frames/input", exist_ok=True)
    return os.system(f"ffmpeg -i {file_path} -r {fps} frames/input/frame_%04d.png")

def remove_outliers(filepath):
    pcd = o3d.io.read_point_cloud(filepath)
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    inlier_cloud = pcd.select_by_index(ind)
    o3d.io.write_point_cloud(filepath, inlier_cloud)

def load_ply(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    return pcd

def draw_normals(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    pcd = pcd.voxel_down_sample(voxel_size=0.005)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(30)
    o3d.io.write_point_cloud("scene_normals.ply", pcd)


def create_mesh(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    
    pts = np.asarray(pcd.points)
    mask = np.isfinite(pts).all(axis=1)
    pcd = pcd.select_by_index(np.where(mask)[0])
    

    if len(pcd.points) > 1000:
        pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=2.0)


    pcd = pcd.voxel_down_sample(0.003)

    
    if len(pcd.points) == 0:
        print("❌ Point cloud empty after cleaning.")
        return None

    
    print("Running Poisson...")
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=10
    )

    if len(mesh.vertices) == 0:
        print("❌ Poisson returned empty mesh.")
        return None

    densities = np.asarray(densities)
    keep_vert_mask = densities > np.quantile(densities, 0.05)  # keep top 95%


    old_verts = np.asarray(mesh.vertices)
    old_tris  = np.asarray(mesh.triangles, dtype=np.int64)

    kept_idx = np.nonzero(keep_vert_mask)[0]
    if kept_idx.size == 0:
        raise RuntimeError("No vertices kept after density thresholding")


    map_old_to_new = -np.ones(old_verts.shape[0], dtype=np.int64)
    map_old_to_new[kept_idx] = np.arange(len(kept_idx), dtype=np.int64)

    tri_mask = np.all(keep_vert_mask[old_tris], axis=1)
    good_tris = old_tris[tri_mask]

    remapped_tris = map_old_to_new[good_tris]

    if (remapped_tris < 0).any() or (remapped_tris.max() >= len(kept_idx)):
        raise RuntimeError("Remapping failed: invalid triangle indices")

    new_verts = old_verts[kept_idx]
    new_mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(new_verts.astype(np.float64)),
        o3d.utility.Vector3iVector(remapped_tris.astype(np.int32))
    )

    new_mesh.remove_unreferenced_vertices()
    new_mesh.compute_vertex_normals()

    print("Filtered mesh:", len(new_mesh.vertices), "verts,", len(new_mesh.triangles), "tris")
    o3d.io.write_triangle_mesh("scene_mesh_poisson_filtered_safe.ply", new_mesh)

def split(file_path):
    mesh = o3d.io.read_triangle_mesh(file_path)
    mesh.remove_unreferenced_vertices()
    mesh.compute_vertex_normals()

    o3d.io.write_triangle_mesh("split/visual.ply", mesh)

    collision = mesh.simplify_quadric_decimation(target_number_of_triangles=1500)
    collision.remove_unreferenced_vertices()
    o3d.io.write_triangle_mesh("split/collision.stl", collision)


In [12]:
remove_outliers("frames/sparse/0/points3D.ply")

In [11]:
draw_normals("frames/sparse/0/points3D.ply")

In [17]:
create_mesh("scene_normals.ply")

Running Poisson...
Filtered mesh: 170907 verts, 335072 tris


In [21]:
split("scene_mesh_poisson_filtered_safe.ply")